In [1]:
# Import all necessary libraries.
import os
import cv2
import numpy as np
import sys
import json
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../')

# specify the path to Mask_RCNN if you placed it outside Nomeroff-net project
MASK_RCNN_DIR = os.path.join(NOMEROFF_NET_DIR, 'Mask_RCNN')

MASK_RCNN_LOG_DIR = os.path.join(NOMEROFF_NET_DIR, 'logs')
OPTIONS_MODEL_PATH =  os.path.join(NOMEROFF_NET_DIR, "models/numberplate_options_2019_05_20.h5")

sys.path.append(NOMEROFF_NET_DIR)

In [2]:
# Import license plate recognition tools.
from NomeroffNet import  filters, RectDetector, TextDetector, OptionsDetector, Detector, textPostprocessing, textPostprocessingAsync

optionsDetector = OptionsDetector({
    "class_region": ["xx_unknown", "eu_ua_2015", "eu_ua_2004", "eu_ua_1995", "eu", "xx_transit", "ru", "kz"]
})
optionsDetector.load(OPTIONS_MODEL_PATH)

Using TensorFlow backend.
W0521 10:54:52.515839 140033328940864 deprecation_wrapper.py:119] From /mnt/data/var/www/html2/js/nomeroff-net_2/NomeroffNet/Detector.py:12: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0521 10:54:52.516434 140033328940864 deprecation_wrapper.py:119] From /mnt/data/var/www/html2/js/nomeroff-net_2/NomeroffNet/Detector.py:14: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0521 10:54:52.797887 140033328940864 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0521 10:54:52.801847 140033328940864 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0521 10:54:52.811716 140033328940864 deprecation_wrapper.py:

In [13]:
def optionChecker(datasetDir, imgFormat="png", partSize=100):
    ann = "ann"
    img = "img"
    annDir = os.path.join(datasetDir, ann)
    imgDir = os.path.join(datasetDir, img)
    
    imgFnames        = []
    predictedRegions = []
    nottedRegions    = []
    nottedCountLines = []
    nottedStates     = []
    zones            = []
    annFnames        = []
    annData          = []
    i = 0
    counter = Counter()
    for dirName, subdirList, fileList in os.walk(annDir):
        for fname in fileList:
            ann_path = os.path.join(annDir, fname)
            
            i += 1
            annFnames.append(ann_path)
            with open(ann_path) as jsonR:
                data = json.load(jsonR)
            img_name = data["name"]
            annData.append(data)
            
            img_path = os.path.join(imgDir, "{}.{}".format(img_name, imgFormat))
            zones.append(cv2.cvtColor(mpimg.imread(img_path), cv2.COLOR_RGB2BGR))
            
            imgFnames.append(img_path)
            nottedRegions.append(data["region_id"])
            nottedStates.append(data["state_id"])
            nottedCountLines.append(data["count_lines"])
            if i >= partSize:
                # find standart
                regionIds, stateIds, countLines = optionsDetector.predict(zones)
                for regionId, stateId, zone, nottedRegion, nottedState, imgFname, annFname, annItem, nottedCountL, countL in zip(regionIds, stateIds, zones, nottedRegions, nottedStates, imgFnames, annFnames, annData, countLines, nottedCountLines):
                    
                    # region
                    bad_region=False
                    if int(regionId) != int(nottedRegion):
                        bad_region=True
                        if not('moderation' in annItem):
                            annItem['moderation'] = {}
                        annItem['moderation']['regionPredicted'] = int(regionId)
                        print("REGION NOT CORRECT IN {}".format(imgFname))
                        print("PREDICTED: {}".format(regionId))
                        print("ANNOTATED: {}".format(nottedRegion))
                        #plt.imshow(cv2.cvtColor(zone, cv2.COLOR_RGB2BGR))
                        #plt.show()
                        counter["BAD_REGION"] += 1
                    else:
                        counter["GOOD_REGION"] += 1
                        
                    # state  
                    bad_state=False
                    if int(stateId) != int(nottedState):
                        bad_state=True
                        if not('moderation' in annItem):
                            annItem['moderation'] = {}
                        annItem['moderation']['statePredicted'] = int(stateId)
                        print("STATE NOT CORRECT IN {}".format(imgFname))
                        print("PREDICTED: {}".format(stateId))
                        print("ANNOTATED: {}".format(nottedState))
                        #plt.imshow(cv2.cvtColor(zone, cv2.COLOR_RGB2BGR))
                        #plt.show()
                        counter["BAD_STATE"] += 1
                    else:
                        counter["GOOD_STATE"] += 1
                    
                    # count
                    bad_count=False
                    if int(countL) != int(nottedCountL):
                        bad_count=True
                        if not('moderation' in annItem):
                            annItem['moderation'] = {}
                        annItem['moderation']['countPredicted'] = int(countL)
                        print("COUNT LINES NOT CORRECT IN {}".format(imgFname))
                        print("PREDICTED: {}".format(countL))
                        print("ANNOTATED: {}".format(nottedCountL))
                        #plt.imshow(cv2.cvtColor(zone, cv2.COLOR_RGB2BGR))
                        #plt.show()
                        counter["BAD_COUNT"] += 1
                    else:
                        counter["GOOD_COUNT"] += 1

                    if 'moderation' in annItem and (bad_region or bad_state or bad_count):
                        with open(annFname, "w") as jsonW:
                            annItem['moderation']['isModerated'] = 0
                            json.dump(annItem, jsonW)                    
                    
                imgFnames        = []
                predictedRegions = []
                nottedRegions    = []
                nottedCountLines = []
                nottedStates     = []
                zones            = []
                annFnames        = []
                annData          = []
    print(counter)

In [16]:
valDir = os.path.join(NOMEROFF_NET_DIR, "datasets/options/val")
optionChecker(valDir)

REGION NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260550992.png
PREDICTED: 7
ANNOTATED: 0
REGION NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260245725.png
PREDICTED: 1
ANNOTATED: 4
REGION NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260104023.png
PREDICTED: 2
ANNOTATED: 3
REGION NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260246245.png
PREDICTED: 1
ANNOTATED: 2
STATE NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260050952.png
PREDICTED: 1
ANNOTATED: 0
REGION NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/261052516.png
PREDICTED: 3
ANNOTATED: 4
REGION NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260248824.png
PREDICTED: 1
ANNOTATED: 4
REGION NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/option_ze

REGION NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260955112.png
PREDICTED: 0
ANNOTATED: 2
STATE NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260040771.png
PREDICTED: 2
ANNOTATED: 1
STATE NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260252008.png
PREDICTED: 0
ANNOTATED: 2
STATE NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260563004.png
PREDICTED: 2
ANNOTATED: 1
REGION NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/261491243.png
PREDICTED: 5
ANNOTATED: 0
STATE NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/261491243.png
PREDICTED: 1
ANNOTATED: 0
COUNT LINES NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/261491243.png
PREDICTED: 0
ANNOTATED: 1
REGION NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/2602313

STATE NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260652059.png
PREDICTED: 2
ANNOTATED: 1
STATE NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260596420.png
PREDICTED: 0
ANNOTATED: 1
STATE NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260264549.png
PREDICTED: 0
ANNOTATED: 2
COUNT LINES NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260264549.png
PREDICTED: 1
ANNOTATED: 0
REGION NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260106449.png
PREDICTED: 1
ANNOTATED: 4
REGION NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260222275.png
PREDICTED: 1
ANNOTATED: 4
REGION NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/260781109.png
PREDICTED: 7
ANNOTATED: 0
STATE NOT CORRECT IN /mnt/data/var/www/html2/js/nomeroff-net_2/datasets/options/val/img/2607811